# Evaluating the results of Training

The results of training (and its evaluation) will depend on how the data was split into training and testing sets. In this worksheet, we use repeated random subsampling to assess the performance of our trained model.

According to [Wikipedia](https://en.wikipedia.org/wiki/Cross-validation_(statistics)):

>This method, also known as Monte Carlo cross-validation,[16] creates multiple random splits of the dataset into training and validation data.[17] For each such split, the model is fit to the training data, and predictive accuracy is assessed using the validation data. The results are then averaged over the splits. The advantage of this method (over k-fold cross validation) is that the proportion of the training/validation split is not dependent on the number of iterations (folds). The disadvantage of this method is that some observations may never be selected in the validation subsample, whereas others may be selected more than once. In other words, validation subsets may overlap. This method also exhibits Monte Carlo variation, meaning that the results will vary if the analysis is repeated with different random splits.

We will be dividing our data into an 80-20 split, using 80% for training and 20% for testing. This will be repeated randomly for each iteration of training to evaluate how much the training improves results on average.


In [1]:
#Import necessary modules

from __future__ import unicode_literals, print_function
import spacy
from spacy.lang.es import Spanish 
from spacy.scorer import Scorer
from spacy.language import GoldParse
from spacy.util import minibatch, compounding

import pandas as pd
import numpy as np
import json
import plac
import random
from sklearn.model_selection import train_test_split
from pathlib import Path
from copy import deepcopy

In [2]:
# Read Tagged Data from JSON file
with open('TaggedData_SF.json', 'r', encoding='utf-8') as fp2:
    TAGGED_DATA = json.load(fp2)

Spacy has a built-in function for evaluating a model's performance using the [command line](https://spacy.io/api/cli#evaluate), but alternatively you can define a function like the one below. It takes the NER model and examples that you input and returns several metrics:
        - UAS (Unlabelled Attachment Score) 
        - LAS (Labelled Attachment Score)
        - ents_p
        - ents_r
        - ents_f
        - tags_acc
        - token_acc

[According](https://github.com/explosion/spaCy/issues/2405) to one of the creators of Spacy, 
>The UAS and LAS are standard metrics to evaluate dependency parsing. UAS is the proportion of tokens whose head has been correctly assigned, LAS is the proportion of tokens whose head has been correctly assigned with the right dependency label (subject, object, etc).
>ents_p, ents_r, ents_f are the precision, recall and fscore for the NER task.
>tags_acc is the POS tagging accuracy.
>token_acc seems to be the precision for token segmentation.

The key metrics for this task are the precision, recall and f-score.
**Precision** (ents_p) is the ratio of correctly-labeled entities out of all the entities labeled. (True Positive/(True Positive+False Positive)).
**Recall**  (ents_r) is the ratio of correctly-labeled entities out of all true entities (True Positive/(True Positive+False Negative)). The F-score is the mean of both values.  

These metrics all appear averaged out through all the entity types (labels) and then detailed for each label in particular. We want these values to be as close as possible to 100. 

In [3]:
 def evaluate(ner_model, examples):
        scorer = Scorer()
        for sents, ents in examples:
            doc_gold = ner_model.make_doc(sents)
            gold = GoldParse(doc_gold, entities=ents['entities'])
            pred_value = ner_model(sents)
            scorer.score(pred_value, gold)
        return scorer.scores

Next, we load the Spacy model, define a blank dataframe to store the output of our different trials, and calculate the amount of data necessary for an 80-20 split. 

In [4]:
# Load Spacy Model
nlp= spacy.load('es_core_news_md')

In [5]:
#Define a blank dataframe with columns for the information we are interested in

columns=['ents_p', 'ents_r', 'ents_f', 'label']
eval_data = pd.DataFrame(columns=columns)
eval_data = eval_data.fillna(0)

In [6]:
# Calculate 80% of data for an 80-20 split

len(TAGGED_DATA)*0.8

326.40000000000003

Finally, we run the training loop ten times, each with a different 80-20 split, and store the evaluation statistics of our NER model in our dataframe. We are using a copy of the NLP model because we want the training to start afresh for each set of training data. Otherwise, the model would be trained on all the data including the test data, leading to the model overperforming on the tagged data compared to new samples that we are interested in tagging later.

In [7]:
# Testing how much the evaluation depends on texts included in testing data

#Loop 10 times
for x in range(0,10):
    
    #Batching the Tagged Data into training and evaluation data (80-20 split)

    random.shuffle(TAGGED_DATA)
    train_data = TAGGED_DATA[:326]
    test_data = TAGGED_DATA[326:]

    #Load the model to be trained (save separately, because we do not want to repeatedly retrain the same model)
    nlp1 = deepcopy(nlp)
    
    #Create object for retrieving the NER pipeline component
    ner=nlp1.get_pipe("ner")

    #Generate new labels for the NER component (if you wish to create new labels)
    ner.add_label("OBJ")
    ner.add_label("MON")
    ner.add_label("DATE")

    #This piece of code creates a loop in which we train the model, but only for the NER component (disabling the tagger and the parser, which we are not using here).
    with nlp1.disable_pipes('tagger','parser'):
    #Here we resume training, alternatively you could begin_training if you are starting on a new model.
        optimizer= nlp1.resume_training()
    #Would need to figure this out, they are the sizes for the minibatching
        sizes = compounding(1.0, 4.0, 1.001)
    #This loops the training mechanism 10 times, randomly shuffling the training data and creating mini-batches from which the algorithm learns to label. Each time a batch is processed, the model is updated.
        for itn in range(10):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp1.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)
    
    #Testing NER results of existing model on test data

    results = evaluate(nlp1,test_data)
    evaluation= dict((k, results[k]) for k in ['ents_per_type'] 
                                        if k in results)
    
    ev_date = [val.get('DATE') for val in evaluation.values()]
    ev_mon= [val.get('MON') for val in evaluation.values()]
    ev_obj= [val.get('OBJ') for val in evaluation.values()]
    ev_org= [val.get('ORG') for val in evaluation.values()]
    ev_per= [val.get('PER') for val in evaluation.values()]
    ev_loc= [val.get('LOC') for val in evaluation.values()]
    
    dlist = list(ev_date[0].values())
    newrow1= {'ents_p': dlist[0],'ents_r': dlist[1],'ents_f':dlist[2],'label':'DATE'}
    
    mlist = list(ev_mon[0].values())
    newrow2= {'ents_p': mlist[0],'ents_r':mlist[1],'ents_f':mlist[2],'label':'MON'}
                  
    oblist = list(ev_obj[0].values())
    newrow3= {'ents_p':oblist[0],'ents_r':oblist[1],'ents_f':oblist[2],'label':'OBJ'}
                  
    orlist = list(ev_org[0].values())
    newrow4= {'ents_p':orlist[0],'ents_r':orlist[1],'ents_f':orlist[2],'label':'ORG'}
                  
    plist = list(ev_per[0].values())
    newrow5= {'ents_p':plist[0],'ents_r':plist[1],'ents_f':plist[2],'label':'PER'}
                  
    llist = list(ev_loc[0].values())
    newrow6= {'ents_p':llist[0],'ents_r':llist[1],'ents_f':llist[2],'label':'LOC'}
                  
    eval_data=eval_data.append(newrow1,ignore_index=True)
    eval_data=eval_data.append(newrow2,ignore_index=True)
    eval_data=eval_data.append(newrow3,ignore_index=True)
    eval_data=eval_data.append(newrow4,ignore_index=True)
    eval_data=eval_data.append(newrow5,ignore_index=True)
    eval_data=eval_data.append(newrow6,ignore_index=True)

/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


Losses {'ner': 28710.784190735434}
Losses {'ner': 26217.361760210915}
Losses {'ner': 25762.10888914067}
Losses {'ner': 25750.571883825527}
Losses {'ner': 25605.796603823735}
Losses {'ner': 25429.72852884233}
Losses {'ner': 25745.145906287013}
Losses {'ner': 25202.404243156314}
Losses {'ner': 25266.99406839907}
Losses {'ner': 25489.240974783897}


/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


Losses {'ner': 31591.14445352634}
Losses {'ner': 28971.689574223412}
Losses {'ner': 28273.77027401668}
Losses {'ner': 28336.130817049503}
Losses {'ner': 28210.249039787763}
Losses {'ner': 27932.240037732292}
Losses {'ner': 27763.095332303084}
Losses {'ner': 27761.657864421606}
Losses {'ner': 27966.925535529852}
Losses {'ner': 27986.069834038615}


/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


...

Losses {'ner': 30254.052561019857}
Losses {'ner': 28162.963799256624}
Losses {'ner': 27546.89418252615}
Losses {'ner': 27500.149581443322}
Losses {'ner': 26897.616367609065}
Losses {'ner': 26974.760558665264}
Losses {'ner': 26839.389122212306}
Losses {'ner': 26972.785244077444}
Losses {'ner': 27031.776955366135}
Losses {'ner': 27015.844194456935}


/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


Losses {'ner': 28270.271258579625}
Losses {'ner': 26794.917620348635}
Losses {'ner': 26116.506573714745}
Losses {'ner': 25864.87159792958}
Losses {'ner': 25542.15608135699}
Losses {'ner': 25435.581236070022}
Losses {'ner': 25549.276633552276}
Losses {'ner': 25479.81967278011}
Losses {'ner': 25041.12965906784}
Losses {'ner': 25443.65072990954}


/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


Losses {'ner': 28671.18886008404}
Losses {'ner': 26576.080714818563}
Losses {'ner': 26100.679036867645}
Losses {'ner': 25881.440064884606}
Losses {'ner': 25993.326108673704}
Losses {'ner': 25671.08909981791}
Losses {'ner': 25842.97500598058}
Losses {'ner': 25603.911143779755}
Losses {'ner': 25803.398500412703}
Losses {'ner': 25493.54890247248}


Below, we print the contents of our evaluation dataframe:

In [8]:
print(eval_data)

        ents_p     ents_r     ents_f label
0    50.000000   5.882353  10.526316  DATE
1    63.414634  32.098765  42.622951   MON
2     0.000000   0.000000   0.000000   OBJ
3    44.444444  23.529412  30.769231   ORG
4    86.142322  80.701754  83.333333   PER
5    80.575540  78.321678  79.432624   LOC
6     0.000000   0.000000   0.000000  DATE
7    64.864865  31.168831  42.105263   MON
8     0.000000   0.000000   0.000000   OBJ
9    15.384615   4.081633   6.451613   ORG
10   85.520362  84.000000  84.753363   PER
11   94.545455  75.362319  83.870968   LOC
12  100.000000   6.666667  12.500000  DATE
13   63.157895  58.536585  60.759494   MON
14    0.000000   0.000000   0.000000   OBJ
15   36.842105  15.217391  21.538462   ORG
16   87.301587  88.000000  87.649402   PER
17   82.170543  76.811594  79.400749   LOC
18    0.000000   0.000000   0.000000  DATE
19   47.058824  35.164835  40.251572   MON
20    0.000000   0.000000   0.000000   OBJ
21   48.000000  18.461538  26.666667   ORG
22   85.865

In [9]:
#Measure mean and standard deviation of f, p and r scores for each label 
a = eval_data.groupby('label').agg({'ents_f':['mean','std'],'ents_p':['mean','std'],'ents_r':['mean','std']})

In [10]:
a

ents_f                ents_p                ents_r           
            mean        std       mean        std       mean        std
label                                                                  
DATE    9.782182   9.578643  41.666667  40.253824   5.821155   5.936515
LOC    80.577654   3.017314  85.263570   4.642661  76.664881   4.973763
MON    55.384011  13.193679  65.566869  13.052479  48.752034  13.813612
OBJ     1.888861   4.417157  23.333333  41.722185   1.003731   2.375146
ORG    22.120174  14.267794  32.923158  19.451804  16.979782  11.556657
PER    85.964481   2.293672  87.336613   2.670296  84.667080   2.563800

#  Evaluating Spelling Normalization

We can apply the evaluation above to a model trained with text whose spelling has been normalized, thus evaluating whether the inclusion of a normalization dictionary improves training results.

In [11]:
# Read Norm Exceptions from JSON file
with open('normalizeddict.json', 'r', encoding='utf-8') as fp3:
    NORM_EXCEPTIONS = json.load(fp3)

In [12]:
#Define and add pipeline component that updates .norm attribute

def add_custom_norms(doc):
    for token in doc:
        if token.text in NORM_EXCEPTIONS:
            token.norm_ = NORM_EXCEPTIONS[token.text]
    return doc

#Add component to the pipeline

nlp.add_pipe(add_custom_norms, first=True)

In [13]:
#Define a new blank dataframe with columns for the information we are interested in

columns=['ents_p', 'ents_r', 'ents_f', 'label']
eval_data2 = pd.DataFrame(columns=columns)
eval_data2 = eval_data2.fillna(0)

In [14]:
# Train and evaluate Model trained with EMS dictionary

#Loop 10 times
for x in range(0,10):
    
    random.shuffle(TAGGED_DATA)
    train_data = TAGGED_DATA[:326]
    test_data = TAGGED_DATA[326:]
    
    #Load the model to be trained
    nlp2 = deepcopy(nlp)
    
    #Create object for retrieving the NER pipeline component
    ner=nlp2.get_pipe("ner")

    #Generate new labels for the NER component (if you wish to create new labels)
    ner.add_label("OBJ")
    ner.add_label("MON")
    ner.add_label("DATE")

    #This piece of code creates a loop in which we train the model, but only for the NER component (disabling the tagger and the parser, which we are not using here).
    with nlp2.disable_pipes('tagger','parser'):
    #Here we resume training, alternatively you could begin_training if you are starting on a new model.
        optimizer= nlp2.resume_training()
    #Would need to figure this out, they are the sizes for the minibatching
        sizes = compounding(1.0, 4.0, 1.001)
    #This loops the training mechanism 10 times, randomly shuffling the training data and creating mini-batches from which the algorithm learns to label. Each time a batch is processed, the model is updated.
        for itn in range(10):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp2.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)
   
 #Testing NER results of existing model on test data

    results = evaluate(nlp2,test_data)
    evaluation= dict((k, results[k]) for k in ['ents_per_type'] 
                                        if k in results)
    
    ev_date = [val.get('DATE') for val in evaluation.values()]
    ev_mon= [val.get('MON') for val in evaluation.values()]
    ev_obj= [val.get('OBJ') for val in evaluation.values()]
    ev_org= [val.get('ORG') for val in evaluation.values()]
    ev_per= [val.get('PER') for val in evaluation.values()]
    ev_loc= [val.get('LOC') for val in evaluation.values()]
    
    dlist = list(ev_date[0].values())
    newrow1= {'ents_p': dlist[0],'ents_r': dlist[1],'ents_f':dlist[2],'label':'DATE'}
    
    mlist = list(ev_mon[0].values())
    newrow2= {'ents_p': mlist[0],'ents_r':mlist[1],'ents_f':mlist[2],'label':'MON'}
                  
    oblist = list(ev_obj[0].values())
    newrow3= {'ents_p':oblist[0],'ents_r':oblist[1],'ents_f':oblist[2],'label':'OBJ'}
                  
    orlist = list(ev_org[0].values())
    newrow4= {'ents_p':orlist[0],'ents_r':orlist[1],'ents_f':orlist[2],'label':'ORG'}
                  
    plist = list(ev_per[0].values())
    newrow5= {'ents_p':plist[0],'ents_r':plist[1],'ents_f':plist[2],'label':'PER'}
                  
    llist = list(ev_loc[0].values())
    newrow6= {'ents_p':llist[0],'ents_r':llist[1],'ents_f':llist[2],'label':'LOC'}
                  
    eval_data2=eval_data2.append(newrow1,ignore_index=True)
    eval_data2=eval_data2.append(newrow2,ignore_index=True)
    eval_data2=eval_data2.append(newrow3,ignore_index=True)
    eval_data2=eval_data2.append(newrow4,ignore_index=True)
    eval_data2=eval_data2.append(newrow5,ignore_index=True)
    eval_data2=eval_data2.append(newrow6,ignore_index=True)

/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


Losses {'ner': 29499.62115381894}
Losses {'ner': 27432.198336651258}
Losses {'ner': 27237.476574885826}
Losses {'ner': 26886.309408007888}
Losses {'ner': 26540.757857780347}
Losses {'ner': 26545.37359688431}
Losses {'ner': 26574.1013068147}
Losses {'ner': 26679.740034505725}
Losses {'ner': 26516.571289405227}
Losses {'ner': 26684.599556565285}


/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


Losses {'ner': 30651.302070253238}
Losses {'ner': 28284.725300171824}
Losses {'ner': 27811.04373792749}
Losses {'ner': 27858.08806299469}
Losses {'ner': 27718.562570926966}
Losses {'ner': 27328.900965396315}
Losses {'ner': 27189.881448478438}
Losses {'ner': 27317.36295453459}
Losses {'ner': 27218.52796754241}
Losses {'ner': 27065.22095376253}


/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


...

Losses {'ner': 31183.07043193781}
Losses {'ner': 28938.966911083655}
Losses {'ner': 28314.097444169314}
Losses {'ner': 27981.325828345012}
Losses {'ner': 27896.63830166764}
Losses {'ner': 27506.291575556505}
Losses {'ner': 27805.07676478289}
Losses {'ner': 27623.085354603827}
Losses {'ner': 27410.957109078765}
Losses {'ner': 27565.289154559374}


/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


Losses {'ner': 29698.50474952518}
Losses {'ner': 28100.42544671885}
Losses {'ner': 27684.143731447766}
Losses {'ner': 27413.299044790547}
Losses {'ner': 27221.78061976153}
Losses {'ner': 27076.371534661856}
Losses {'ner': 26987.433071333915}
Losses {'ner': 27206.584793627262}
Losses {'ner': 26959.954725861317}
Losses {'ner': 27405.67962694168}


/Users/Felipe/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W020] Unnamed vectors. This won't allow multiple vectors models to be loaded. (Shape: (20000, 50))
  "__main__", mod_spec)


Losses {'ner': 27282.973104532}
Losses {'ner': 25546.693975118553}
Losses {'ner': 24631.723043808382}
Losses {'ner': 24399.073844784987}
Losses {'ner': 24632.41723647827}
Losses {'ner': 24773.259103098884}
Losses {'ner': 24214.003735827282}
Losses {'ner': 24327.542434114963}
Losses {'ner': 23951.511757960543}
Losses {'ner': 24057.59735112358}


In [15]:
b= eval_data2.groupby('label').agg({'ents_f':['mean','std'],'ents_p':['mean','std'],'ents_r':['mean','std']})

Below, we print the statistics for the training with (b) and without (a) spelling normalization. As can be seen, there is a slight improvement on most measurements (as well as a reduction in variability) when we normalize spelling. 

In [16]:
a

ents_f                ents_p                ents_r           
            mean        std       mean        std       mean        std
label                                                                  
DATE    9.782182   9.578643  41.666667  40.253824   5.821155   5.936515
LOC    80.577654   3.017314  85.263570   4.642661  76.664881   4.973763
MON    55.384011  13.193679  65.566869  13.052479  48.752034  13.813612
OBJ     1.888861   4.417157  23.333333  41.722185   1.003731   2.375146
ORG    22.120174  14.267794  32.923158  19.451804  16.979782  11.556657
PER    85.964481   2.293672  87.336613   2.670296  84.667080   2.563800

In [17]:
b

ents_f               ents_p                ents_r           
            mean       std       mean        std       mean        std
label                                                                 
DATE    6.181599  9.238110  21.666667  33.379598   3.616384   5.377218
LOC    82.013233  2.651031  86.110284   4.359660  78.437365   3.290363
MON    51.947078  9.837542  70.758458  12.463683  42.015463  10.894793
OBJ     2.933807  2.513359  48.333333  41.163630   1.525103   1.316534
ORG    22.247787  8.778882  31.986232  12.151958  17.145127   6.982109
PER    86.558603  2.209405  88.364413   2.650517  84.850763   2.290407